# Amazon Bedrock Claude usecases for Indian Languages

*This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

*The sample content used in this notebook are purely for learning, informational and demonstration purposes only.*


---
## Claude Model Selection

We have following Claude 3 version of models available as of today on Amazon Bedrock

| Model Name | Bedrock API model name |
|--|--|
| Claude 3 Haiku | anthropic.claude-3-haiku-20240307-v1:0 |
| Claude 3 Sonnet | anthropic.claude-3-sonnet-20240229-v1:0 |
| Claude 3 Opus | Coming Soon |

- [Anthropic Claude Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html#model-parameters-anthropic-claude-messages-overview)
- [Create a Message](https://docs.anthropic.com/claude/reference/messages_post)
- [Amazon Bedrock API for Claude](https://docs.anthropic.com/claude/reference/claude-on-amazon-bedrock)
- [Amazon Bedrock Pricing Page](https://aws.amazon.com/bedrock/pricing/)
---

In [2]:
claude3_haiku_id = "anthropic.claude-3-haiku-20240307-v1:0"
claude3_sonnet_id = "anthropic.claude-3-sonnet-20240229-v1:0"

DEFAULT_MODEL = claude3_haiku_id

In [3]:
import boto3
import json

class LLM:
    def __init__(self, model_id):
        self.model_id = model_id
        self.bedrock = boto3.client(service_name="bedrock-runtime")
        
    def invoke(self, system, messages, max_tokens=128):
        body = json.dumps({
                  "max_tokens": max_tokens,
                  "system": system,
                  "messages": messages,
                  "anthropic_version": "bedrock-2023-05-31"
                })
        response = self.bedrock.invoke_model(
                                            body=body, 
                                            modelId=self.model_id)

        response_body = json.loads(response.get("body").read())
        return response_body['content']
    
llm = LLM(DEFAULT_MODEL)

In [4]:
def system_prompt(language):
    system_prompt = f"""
      Hello, You are an expert in {language}, an Indian language. 
      Given the content in {language} wrapped between <{language}> and </{language}> tags, there will be follow up questions asked to you in English between <question> and </question> tags. You respond in {language} language. You respond to the point and not verbose. Your response will be between the tags <response> and </response>. For responses that requires to identify entities, you just include those entities and nothing more verbose in the response tags.
      You are not verbose and your response doesn't include response out of context given to you. 
      If the question doesn't have a relevant answer in the givent content, you must answer "I don't have an answer" translated in {language}.
      """

    return system_prompt

def actual_message_builder(content, language, question):
    actual_message = [
                        {"role": "user", "content": f"""<{language}>{content}</{language}>
                        <question>{question}</question>"""}
                    ]
    return actual_message

def ask_question(language, language_examples, content, question):
    actual_message = actual_message_builder(content, language, question)
    message = language_examples + actual_message
    response_text = llm.invoke(
        system_prompt(language),
        message,
        max_tokens=1500,
    )
    if response_text:
        print(response_text[0].get('text'))

## Hindi Examples

In [5]:
language = 'Hindi'
# few shot examples
hindi_examples = [
    {"role": "user", "content": f"""<{language}>लखनऊ के प्रसिद्ध चंद्रिका देवी मंदिर की महिमा और इतिहास बेहद अद्भुत है. मंदिर रामायण और महाभारत काल से जुड़ा हुआ है और यहाँ की वर्तमान शुरुआत 300 साल पुरानी है. देशभर से भक्त यहाँ अपनी मनोकामना लेकर आते हैं और देवी की शक्ति को नमन करते हैं. देखें वीडियो.</{language}>
    <question>Any location info present in the news?</question>"""},
    {"role": "assistant", "content": "<response>लखनऊ</response>"},
    {"role": "user", "content": f"""<question>What is famous in this news?</question>"""},
    {"role": "assistant", "content": "<response>चंद्रिका देवी मंदिर की महिमा और इतिहास</response>"},
]

In [6]:
content = """सारा अली खान स्टारर 'ऐ वतन मेरे वतन' अमेजन प्राइम पर स्ट्रीम हुई है. जो कि एक बायोपिक है. फिल्म की कहानी क्रांतिकारी उषा मेहता के जीवन पर आधारित है,  जिन्हें एक सच्ची गांधीवादी और कांग्रेस रेडियो की फाउंडर के रूप में भी जाना जाता है. बताते हैं कैसी है ये फिल्म और किरदार को जीवंत करने में सारा अली खान कितनी खरी उतर पाईं."""

In [7]:
question = "What is the content about?"
ask_question(language, hindi_examples, content, question)

<response>सारा अली खान अभिनीत 'ऐ वतन मेरे वतन' बायोपिक फिल्म के बारे में</response>


In [8]:
question = "Who is the person being discussed about?"
ask_question(language, hindi_examples, content, question)

<response>उषा मेहता</response>


In [9]:
question = "Who is the actor being discussed about?"        
ask_question(language, hindi_examples, content, question)

<response>सारा अली खान</response>


In [10]:
question = """
list all the named entities and their nature such as person, moviename, place, city, type etc in the format 
{'ent1': 'person', 'ent2': 'person', 'ent2': 'city','ent4': 'place', 'ent5':'celebrity'}
"""
ask_question(language, hindi_examples, content, question)

<response>
{'सारा अली खान': 'व्यक्ति', 'ऐ वतन मेरे वतन': 'फिल्म का नाम', 'उषा मेहता': 'व्यक्ति', 'गांधीवादी': 'व्यक्तित्व', 'कांग्रेस रेडियो': 'संस्था'}
</response>


In [11]:
question = "Which animal is mentioned?"        
ask_question(language, hindi_examples, content, question)

<response>मैं नहीं जानता, इस खबर में किसी जानवर का उल्लेख नहीं है</response>


## Telugu Examples

In [12]:
language = 'Telugu'

In [13]:
content = """
ఎన్‌పీసీఐకు సంబంధించిన తాజా ప్రకటనలో క్రెడిట్ కార్డు ద్వారా ఈఎంఐ సదుపాయం, క్రెడిట్ ఖాతా బిల్లు చెల్లింపు, వాయిదా చెల్లింపు ఎంపికలు, పరిమితి నిర్వహణ కార్యాచరణల వంటి అనేక కీలక సదుపాయాలను అందిస్తున్నాయి. 
మే 31, 2024లోపు ఈ ఫీచర్లను అమలు చేయాలని బ్యాంకులు, కార్డు జారీచేసే వారితో సహా జారీ చేసే సంస్థలు ఆదేశించింది.
జూన్ 30 2024 లోగా అన్ని వివరాలు ఆధార్ కార్డ్ లొ పొందు పరచాలి.
"""
telugu_examples = []

In [14]:
question = "What is the content about?"
ask_question(language, telugu_examples, content, question)

<response>ఎన్పీసీఐ కొత్త ప్రకటన ప్రకారం క్రెడిట్ కార్డుల చెల్లింపు సదుపాయాలు, పరిమితి నిర్వహణ, ఆధార్ కార్డు వివరాలను పొందుపరచాల్సిందిగా ఉంది</response>


In [15]:
question = "Who is the person being discussed about?"
ask_question(language, telugu_examples, content, question)

<response>ఎలాంటి వ్యక్తి గురించి చర్చించలేదు. ఇది ఎన్పిసిఐ గురించి క్రెడిట్ కార్డుల ద్వారా చెల్లింపు సదుపాయాలు, వాటిని అమలు చేయాల్సిన గడువుల గురించి సమాచారం ఇవ్వబడింది</response>


In [16]:
question = "Who is the actor being discussed about?"        
ask_question(language, telugu_examples, content, question)

<response>నేను డీఆర్బీకి సంబంధించిన ఏ సమాచారాన్నీ లేదా నటుడి గురించి సమాచారాన్నీ కనుగొనలేదు, కావున మీ ప్రశ్నకు సమాధానం చెప్పలేను.</response>


In [17]:
question = "Is there any benefit described in news?"        
ask_question(language, telugu_examples, content, question)

<response>అవును, క్రెడిట్ కార్డ్ ద్వారా ఈఎంఐ సదుపాయం, క్రెడిట్ ఖాతా బిల్లు చెల్లింపు, వాయిదా చెల్లింపు ఎంపికలు, పరిమితి నిర్వహణ కార్యాచరణలను అందిస్తున్నారని చెప్పారు</response>


In [18]:
question = "Any dates or deadlines? respond with {'DD-MM-YYYY': [action1, action2, ..], 'DD-MM-YYY': [action1]}"        
ask_question(language, telugu_examples, content, question)

<response>
{'31-05-2024': ['అనుసరించవలసిన కీలక సదుపాయాలను అమలు చేయాలి'], '30-06-2024': ['అన్ని వివరాలు ఆధార్ కార్డ్లో పొందుపరచాలి']}
</response>
